In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

### 任意地点の補正量を計算する関数

In [2]:

def bilinear_interpolation(lontarget, lattarget, dH, lonmin, latmin, dlon, dlat):
    ilonf = (lontarget - lonmin)/dlon
    ilatf = (lattarget - latmin)/dlat

    # 切り捨てしたインデックス（南西点）を取得
    ilont = np.floor(ilonf).astype(int)
    ilatt = np.floor(ilatf).astype(int)

    # セル内の相対的な位置(0-1)
    u = ilonf - ilont
    v = ilatf - ilatt
    # 各点の補正量
    f_sw = dH[ilont, ilatt]  # 南西点
    f_se = dH[ilont + 1, ilatt]  # 南東点
    f_nw = dH[ilont, ilatt + 1]  # 北西点
    f_ne = dH[ilont + 1, ilatt + 1]  # 北東点

    # nanがある場合は補間できない
    if np.isnan([f_sw, f_se, f_nw, f_ne]).any():
        print("補間できない点があります。")
        res = np.nan
    else:
        # 2. バイリニア補間公式
        # f(u,v) = (1-u)(1-v)f00 + u(1-v)f10 + (1-u)vf01 + uvf11
        res = (1 - u) * (1 - v) * f_sw + \
              u * (1 - v) * f_se + \
              (1 - u) * v * f_nw + \
              u * v * f_ne

    return res

### パラメータの設定

In [3]:
### 3次メッシュコードの緯度経度の長さを計算
lat_unit = 2 / 3  # degrees
lon_unit = 1.0

dlat = (lat_unit / 8 / 10) 
dlon = (lon_unit / 8 / 10) 

print("dlon, dlat:", dlon*3600, dlat*3600, "second")

### パラメータファイルを二次元メッシュに変換
gdf = gpd.read_parquet("hyokorevBM_jgd2024_h.parquet")
dH = gdf['dH(m)'].values
lon = gdf.geometry.x.values
lat = gdf.geometry.y.values

nx = int(round((lon.max() - lon.min())/dlon))+1
ny = int(round((lat.max() - lat.min())/dlat))+1

print("number of x, y:", nx, ny)

ilon = np.round((lon - lon.min())/dlon).astype(int)
ilat = np.round((lat - lat.min())/dlat).astype(int)
dHmatrix = np.full((int(nx), int(ny)), np.nan, dtype=np.float32)
dHmatrix[ilon, ilat] = dH

lonmin, latmin = lon.min(), lat.min()

dlon, dlat: 45.0 30.0 second
number of x, y: 2491 2603


### 任意地点の補正量を計算する

In [4]:
lattarget = 36.103774791666666
lontarget = 140.08785504166664
r = bilinear_interpolation(lontarget, lattarget, dHmatrix, lonmin, latmin, dlon, dlat)
print(f"補正量: {r:.6f} m")

x = 23.619 - 23.64
print(f"地理院Web版PatchJGD(標高版)の変換値:{x:.6f} m")

補正量: -0.021165 m
地理院Web版PatchJGD(標高版)の変換値:-0.021000 m


### 一括実行

In [5]:
# read input file Web版PatchJGD(標高版) の出力ファイル
path = r"..\xy_h.out"
df = pd.read_csv(path, skiprows=10, sep="\\s+", names=['x', 'y', 'hin','hout'])
# x,yを緯度経度に変換※x,yの順番に注意：地理院サイトはGISソフトウェア等と逆になる
gdfin = gpd.GeoDataFrame(df[['hin','hout']], geometry=gpd.points_from_xy(df['y'].astype(float), df['x'].astype(float)), crs='EPSG:6678')
gdfin2 = gdfin.to_crs('EPSG:6668')
lons, lats = gdfin2.geometry.x, gdfin2.geometry.y

# 補正量を計算
results = []
for lontarget, lattarget in zip(lons, lats):
    r = bilinear_interpolation(lontarget, lattarget, dHmatrix, lonmin, latmin, dlon, dlat)
    print(f"補正量: {r:.6f} m")
    results.append(r)

results = np.array(results)

補正量: -0.211490 m
補正量: -0.211631 m
補正量: -0.211742 m
補正量: -0.211729 m
補正量: -0.211910 m
補正量: -0.211552 m
補正量: -0.211452 m
補正量: -0.211358 m
補正量: -0.211195 m
補正量: -0.211189 m
補正量: -0.211131 m
補正量: -0.211140 m
補正量: -0.211154 m
補正量: -0.211170 m
補正量: -0.211203 m
補正量: -0.211292 m
補正量: -0.211386 m
補正量: -0.211470 m
補正量: -0.211503 m
補正量: -0.211524 m
補正量: -0.211584 m
補正量: -0.211660 m
補正量: -0.211754 m
補正量: -0.211823 m
補正量: -0.211886 m
補正量: -0.212003 m
補正量: -0.212148 m
補正量: -0.212334 m
補正量: -0.212528 m
補正量: -0.212671 m
補正量: -0.212585 m
補正量: -0.212526 m
補正量: -0.212434 m
補正量: -0.212369 m
補正量: -0.212596 m
補正量: -0.212978 m
補正量: -0.213321 m
補正量: -0.213646 m
補正量: -0.214043 m
補正量: -0.214254 m
補正量: -0.214488 m
補正量: -0.214690 m
補正量: -0.214776 m
補正量: -0.214784 m
補正量: -0.214969 m
補正量: -0.215432 m
補正量: -0.215893 m
補正量: -0.216298 m
補正量: -0.216853 m
補正量: -0.217339 m
補正量: -0.217310 m
補正量: -0.216987 m
補正量: -0.216701 m
補正量: -0.216404 m
補正量: -0.216266 m
補正量: -0.216098 m
補正量: -0.216026 m
補正量: -0.215973 m
補正量: -0.215972

In [6]:
dhc = gdfin2['hout'].values - gdfin2['hin'].values
xx = results - dhc
print("平均誤差:", np.nanmean(xx), "最大誤差:", np.nanmax(xx), "最小誤差:", np.nanmin(xx))

平均誤差: 5.7685542693743294e-06 最大誤差: 0.0004983876454069114 最小誤差: -0.0004998050246612651
